# Deliverable

This notebook is adapted from `quickstart_local.ipynb` and contains the following modifications:

In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4' # Set to the index of the GPU you want to use; see visible GPUs with `nvidia-smi` on command line

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from hypothesaes.quickstart import train_sae, interpret_sae, generate_hypotheses, evaluate_hypotheses
from hypothesaes.embedding import get_local_embeddings
from hypothesaes.llm_local import get_vllm_engine

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Datasets
The two datasets I will be using is [Twitter Tweet Sentiments (27.5k)](https://www.kaggle.com/datasets/yasserh/twitter-tweets-sentiment-dataset), a collection of 27,481 tweets and the associated sentiments. 2748 of these tweets are used for validation during SAE training and 2748 tweets used for holdout evaluation. The target variable is the `sentiment` column, which can be `negative`, `neutral`, or `positive` and we are interested in seeing what features of the `text` column predict it.

In [14]:
current_dir = os.getcwd()
assert current_dir.endswith("phoenix")

base_dir = os.path.join("data")
df = pd.read_csv(os.path.join(base_dir, "Tweets.csv"))

train_df, val_df = train_test_split(df, test_size=2748*2, train_size=21984, random_state=42)
val_df_SAE, val_df_holdout = train_test_split(val_df, test_size=2748, train_size=2748, random_state=42)

train_df.head(5)

,textID,text,selected_text,sentiment
16604,9efc18dfd9,watching a movie and rocking my baby kitty who...,watching a movie and rocking my baby kitty who...,positive
11293,7aef27c516,Doctor Who has finished,Doctor Who has finished,neutral
11299,415660cb0e,you should.,you should.,neutral
18204,4fdc228bbe,"back at school again. almost weekend. oh wait,...","back at school again. almost weekend. oh wait,...",neutral
22728,ea91e5a7ab,My computer is SO slooowww this morning. I th...,My computer is SO slooowww this morning. I th...,neutral
